#Libraries and function for preprocessing

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

In [ ]:
def scale_dataset(dataframe, oversample = False ):  #default value
  X = dataframe[dataframe.columns[:-1]].values
  y = dataframe[dataframe.columns[-1]].values

  scaler = StandardScaler()
  X = scaler.fit_transform(X)

  if oversample :
    ros = RandomOverSampler()
    X,y = ros.fit_resample(X,y) #Take more of the class with less data samples and keep sampling them

  data = np.hstack((X, np.reshape(y, (-1,1))))

  return data, X, y

#Rice

Data : https://archive.ics.uci.edu/dataset/545/rice+cammeo+and+osmancik

In [ ]:
cols = ["Area Integer", "Perimeter Real", "Major_Axis_Length Real",
        "Minor_Axis_Length Real", "Eccentricity	Real" ,"Convex_Area	Integer", "Extent Real", "Class" ]

In [ ]:
df = pd.read_csv("rice.csv", names = cols)


In [ ]:
df['Class'] = (df['Class'] == "Cammeo").astype(int)
df['Class'].unique()

array([1, 0])

In [ ]:
train, valid, test = np.split(df.sample(frac=1), [int(0.6*len(df)) , int(0.8*len(df))])


In [ ]:
train, Xtrain, ytrain = scale_dataset(train, True)
valid, Xvalid, yvalid = scale_dataset(valid, False)
test, Xtest, ytest = scale_dataset(test, False)

In [ ]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [ ]:
knnModel = KNeighborsClassifier(n_neighbors=5)
knnModel.fit(Xtrain,ytrain)
ypredknn = knnModel.predict(Xtest)
print(classification_report(ytest,ypredknn))

              precision    recall  f1-score   support

           0       0.92      0.90      0.91       428
           1       0.87      0.90      0.89       334

    accuracy                           0.90       762
   macro avg       0.90      0.90      0.90       762
weighted avg       0.90      0.90      0.90       762



In [ ]:
#NB
from sklearn.naive_bayes import GaussianNB

In [ ]:
nbModel = GaussianNB()
nbModel.fit(Xtrain,ytrain)
yprednb = nbModel.predict(Xtest)
print(classification_report(ytest,yprednb))

              precision    recall  f1-score   support

           0       0.91      0.94      0.93       428
           1       0.92      0.88      0.90       334

    accuracy                           0.92       762
   macro avg       0.92      0.91      0.91       762
weighted avg       0.92      0.92      0.92       762



In [ ]:
#LR
from sklearn.linear_model import LogisticRegression

In [ ]:
lrModel = LogisticRegression()
lrModel.fit(Xtrain,ytrain)
ypredlr = lrModel.predict(Xtest)
print(classification_report(ytest, ypredlr))

              precision    recall  f1-score   support

           0       0.93      0.93      0.93       428
           1       0.92      0.91      0.91       334

    accuracy                           0.93       762
   macro avg       0.92      0.92      0.92       762
weighted avg       0.93      0.93      0.93       762



In [ ]:
#SVM
from sklearn.svm import SVC

In [ ]:
svModel = SVC()
svModel.fit(Xtrain,ytrain)
ypredsv = svModel.predict(Xtest)
print(classification_report(ytest,ypredsv))

              precision    recall  f1-score   support

           0       0.93      0.93      0.93       428
           1       0.90      0.91      0.91       334

    accuracy                           0.92       762
   macro avg       0.92      0.92      0.92       762
weighted avg       0.92      0.92      0.92       762



#Breast Cancer

Data : https://archive.ics.uci.edu/dataset/15/breast+cancer+wisconsin+original

In [ ]:
cols = ["Sample code number",
        "Clump Thickness",
        "Uniformity of Cell Size",
        "Uniformity of Cell Shape",
        "Marginal Adhesion",
        "Single Epithelial Cell Size",
        "Bare Nuclei",
        "Bland Chromatin",
        "Normal Nucleoli",
        "Mitoses",
        "Class"]

In [ ]:
df = pd.read_csv("breast-cancer-wisconsin.data", names = cols)
df.pop('Sample code number')

0      1000025
1      1002945
2      1015425
3      1016277
4      1017023
        ...   
678     776715
679     841769
680     888820
681     897471
682     897471
Name: Sample code number, Length: 683, dtype: int64

In [ ]:
df['Class'] = (df['Class'] == 4).astype(int)
df.head()

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,5,1,1,1,2,1,3,1,1,0
1,5,4,4,5,7,10,3,2,1,0
2,3,1,1,1,2,2,3,1,1,0
3,6,8,8,1,3,4,3,7,1,0
4,4,1,1,3,2,1,3,1,1,0


In [ ]:
train, test = np.split(df.sample(frac=1), [int(0.8*len(df))])

In [ ]:
train, Xtrain, ytrain = scale_dataset(train, True)
test, Xtest, ytest = scale_dataset(test, False)

In [ ]:
import tensorflow as tf

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation = tf.keras.activations.relu, input_shape = (9,)),
    tf.keras.layers.Dense(128,activation = tf.keras.activations.relu),
    tf.keras.layers.Dense(1,activation = tf.keras.activations.sigmoid)
    #tf.keras.layers.Dense(2,activation = tf.keras.activations.softmax)
])

model.compile(optimizer = 'adam',
              #loss = tf.keras.losses.sparse_categorical_crossentropy,
              loss = tf.keras.losses.binary_crossentropy,
              metrics = ['accuracy']
             )

#Can't use binary cross entropy if output layer is more than 1 neuron. if using more than one sparse_categorical_crossentropy
#Softmax will work for more than one neuron only cannot be 1. If using 1 neuron use sigmoid

In [ ]:
model.fit(Xtrain,ytrain,epochs = 10)

Epoch 1/10
23/23 [==============================] - 1s 4ms/step - loss: 0.3739 - accuracy: 0.9100
Epoch 2/10
23/23 [==============================] - 0s 4ms/step - loss: 0.1023 - accuracy: 0.9751
Epoch 3/10
23/23 [==============================] - 0s 4ms/step - loss: 0.0757 - accuracy: 0.9765
Epoch 4/10
23/23 [==============================] - 0s 4ms/step - loss: 0.0691 - accuracy: 0.9765
Epoch 5/10
23/23 [==============================] - 0s 4ms/step - loss: 0.0649 - accuracy: 0.9792
Epoch 6/10
23/23 [==============================] - 0s 4ms/step - loss: 0.0617 - accuracy: 0.9820
Epoch 7/10
23/23 [==============================] - 0s 4ms/step - loss: 0.0590 - accuracy: 0.9834
Epoch 8/10
23/23 [==============================] - 0s 4ms/step - loss: 0.0568 - accuracy: 0.9848
Epoch 9/10
23/23 [==============================] - 0s 5ms/step - loss: 0.0544 - accuracy: 0.9861
Epoch 10/10
23/23 [==============================] - 0s 4ms/step - loss: 0.0521 - accuracy: 0.9848


In [ ]:
model.evaluate(Xtest,ytest)

5/5 [==============================] - 0s 4ms/step - loss: 0.0765 - accuracy: 0.9708


[0.07647543400526047, 0.970802903175354]

In [ ]:
ytrain.shape

(722,)

In [ ]:
ytest.shape

(137,)

#Fungi Image Processing

Data : https://archive.ics.uci.edu/dataset/773/defungi

In [ ]:
import tensorflow as tf

In [ ]:
!unzip /content/drive/MyDrive/defungi.zip  #Puts it in the content folder by default also

In [ ]:
ds = tf.keras.utils.image_dataset_from_directory(
    'defungi',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(224, 224),
)

Found 9114 files belonging to 5 classes.


In [ ]:
import tensorflow_datasets as tfds

In [ ]:
ds

<_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [ ]:
from keras.layers.pooling.max_pooling2d import MaxPool2D
#Making a CNN

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32,(3,3),activation = 'relu', input_shape = (224,224,3)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64,(3,3),activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64,(3,3),activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64,activation = 'relu'),
    tf.keras.layers.Dense(6,activation = tf.keras.activations.softmax)
])


model.compile(optimizer = 'adam',
              loss = tf.keras.losses.sparse_categorical_crossentropy,
              metrics = ['accuracy'])

In [ ]:
model.fit(ds,epochs = 5)

Epoch 1/5
285/285 [==============================] - 23s 79ms/step - loss: 1.3378 - accuracy: 0.4833
Epoch 2/5
285/285 [==============================] - 18s 64ms/step - loss: 1.3373 - accuracy: 0.4832
Epoch 3/5
285/285 [==============================] - 19s 65ms/step - loss: 1.3440 - accuracy: 0.4832
Epoch 4/5
285/285 [==============================] - 20s 67ms/step - loss: 1.4314 - accuracy: 0.4816
Epoch 5/5
285/285 [==============================] - 19s 64ms/step - loss: 1.3375 - accuracy: 0.4832


##Pretrained CNN

In [ ]:
IMG_SHAPE = (224, 224, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False

In [ ]:
global_average_pooling = tf.keras.layers.GlobalAveragePooling2D() #For flattening the data that base model will forward
prediction_layer = tf.keras.layers.Dense(6,activation = 'softmax')

In [ ]:
model_pretrained = tf.keras.Sequential([
    base_model,
    global_average_pooling,
    tf.keras.layers.Dense(64,activation = 'relu'),
    prediction_layer
])

In [ ]:
model_pretrained.compile(optimizer = 'adam',
              loss = tf.keras.losses.sparse_categorical_crossentropy,
              metrics = ['accuracy'])

In [ ]:
model_pretrained.fit(ds,epochs = 10)

Epoch 1/10
285/285 [==============================] - 23s 64ms/step - loss: 0.9540 - accuracy: 0.6062
Epoch 2/10
285/285 [==============================] - 18s 62ms/step - loss: 0.8021 - accuracy: 0.6696
Epoch 3/10
285/285 [==============================] - 19s 66ms/step - loss: 0.7519 - accuracy: 0.6914
Epoch 4/10
285/285 [==============================] - 19s 63ms/step - loss: 0.7124 - accuracy: 0.7075
Epoch 5/10
285/285 [==============================] - 17s 60ms/step - loss: 0.6810 - accuracy: 0.7222
Epoch 6/10
285/285 [==============================] - 18s 61ms/step - loss: 0.6468 - accuracy: 0.7385
Epoch 7/10
285/285 [==============================] - 19s 67ms/step - loss: 0.6294 - accuracy: 0.7415
Epoch 8/10
285/285 [==============================] - 18s 61ms/step - loss: 0.6078 - accuracy: 0.7535
Epoch 9/10
285/285 [==============================] - 18s 62ms/step - loss: 0.5764 - accuracy: 0.7673
Epoch 10/10
285/285 [==============================] - 19s 67ms/step - loss: 0.558

Achieving 48 percent accuracy with my own covnet and the pretrained covnet reaches upto 77. So not too shabby :)

#Drowsiness Detection

Data : https://www.kaggle.com/datasets/hazemfahmy/openned-closed-eyes

In [ ]:
import tensorflow as tf

In [ ]:
!unzip /content/drive/MyDrive/TrainingData/Drowsiness/TestSet.zip
!unzip /content/drive/MyDrive/TrainingData/Drowsiness/TrainingSet.zip

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    'TrainingSet',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(224, 224),
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    'TestSet',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(224, 224),
)

Found 1704 files belonging to 2 classes.
Found 4232 files belonging to 2 classes.


In [ ]:
train_ds


<_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32,(3,3),activation = 'relu', input_shape = (224,224,3)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64,(3,3),activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64,(3,3),activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64,activation = 'relu'),
    tf.keras.layers.Dense(2,activation = tf.keras.activations.softmax)
])


model.compile(optimizer = 'adam',
              loss = tf.keras.losses.sparse_categorical_crossentropy,
              metrics = ['accuracy'])

In [ ]:
model.fit(train_ds,epochs = 5)

Epoch 1/5
54/54 [==============================] - 6s 72ms/step - loss: 15.6583 - accuracy: 0.7142
Epoch 2/5
54/54 [==============================] - 4s 68ms/step - loss: 0.2462 - accuracy: 0.9120
Epoch 3/5
54/54 [==============================] - 4s 60ms/step - loss: 0.1484 - accuracy: 0.9542
Epoch 4/5
54/54 [==============================] - 4s 60ms/step - loss: 0.1102 - accuracy: 0.9619
Epoch 5/5
54/54 [==============================] - 5s 90ms/step - loss: 0.0690 - accuracy: 0.9759


In [ ]:
model.evaluate(test_ds)

133/133 [==============================] - 7s 52ms/step - loss: 0.2027 - accuracy: 0.9279


[0.20272010564804077, 0.9279300570487976]

WOW

In [ ]:
IMG_SHAPE = (224, 224, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False

global_average_pooling = tf.keras.layers.GlobalAveragePooling2D() #For flattening the data that base model will forward
prediction_layer = tf.keras.layers.Dense(2,activation = 'softmax')

In [ ]:
model_pretrained = tf.keras.Sequential([
    base_model,
    global_average_pooling,
    tf.keras.layers.Dense(64,activation = 'relu'),
    prediction_layer
])

model_pretrained.compile(optimizer = 'adam',
              loss = tf.keras.losses.sparse_categorical_crossentropy,
              metrics = ['accuracy'])

In [ ]:
model_pretrained.fit(train_ds,epochs = 5)

Epoch 1/5
54/54 [==============================] - 7s 58ms/step - loss: 0.3503 - accuracy: 0.8521
Epoch 2/5
54/54 [==============================] - 5s 87ms/step - loss: 0.2033 - accuracy: 0.9196
Epoch 3/5
54/54 [==============================] - 5s 81ms/step - loss: 0.1784 - accuracy: 0.9313
Epoch 4/5
54/54 [==============================] - 3s 57ms/step - loss: 0.1895 - accuracy: 0.9284
Epoch 5/5
54/54 [==============================] - 5s 77ms/step - loss: 0.1698 - accuracy: 0.9349


In [ ]:
model_pretrained.evaluate(test_ds)

133/133 [==============================] - 9s 59ms/step - loss: 0.1875 - accuracy: 0.9293


[0.1874774545431137, 0.929347813129425]

Not bad. Almost as good as pretrained. 92.7 vs 92.9